In [ ]:
import sys
sys.path.append("/workdir/unsupervised_pretrain/")

In [ ]:
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm
import numpy as np
from models import SeriesResNet18
from datasets import SeriesEmbedDataset

# Get ready to do some business #

In [ ]:
ds = SeriesEmbedDataset(["/datasets/datasets/unsupervised-sentinel2/testset-16SEF/"], size=512, series_length=20)

In [ ]:
ds = SeriesEmbedDataset(["/datasets/datasets/berlin/32UQD/"], size=512, series_length=8)
print(len(ds))

# Load the test set, compute embeddings, save embeddings #

This only needs to be done once.

In [ ]:
device = torch.device("cuda")

In [ ]:
model = torch.load("/workdir/unsupervised_pretrain/model.pth", map_location=device).to(device)
model = model.eval()
autoencoder = torch.load("/workdir/unsupervised_pretrain/model-autoencoder.pth", map_location=device).to(device)
autoencoder = autoencoder.eval()

In [ ]:
visual_embeddings = []
text_embeddings = []
skip = 2

with torch.inference_mode():
    for i in tqdm(range(0, len(ds), skip)):
        imagery, _, text_embedding = ds[i]
        imagery = torch.unsqueeze(imagery.to(device), dim=0)

        visual_embedding = model(imagery)
        visual_embedding = F.normalize(visual_embedding, dim=1)

        text_embedding = torch.unsqueeze(torch.from_numpy(text_embedding), dim=0).to(device)
        text_embedding = F.normalize(text_embedding, dim=1)

        visual_embedding = visual_embedding.detach().cpu()
        text_embedding = text_embedding.detach().cpu()

        visual_embeddings.append(visual_embedding)
        text_embeddings.append(text_embedding)

In [ ]:
text_embeddings = torch.cat(text_embeddings, dim=0)

In [ ]:
torch.save(text_embeddings, "/workdir/unsupervised_pretrain/jupyter/text-embeddings.t")

In [ ]:
visual_embeddings = torch.cat(visual_embeddings, dim=0)

In [ ]:
torch.save(visual_embeddings, "/workdir/unsupervised_pretrain/jupyter/visual-embeddings.t")

In [ ]:
with torch.inference_mode():
    stuff = autoencoder(F.normalize(visual_embeddings.to(device), dim=1), text_embeddings.to(device))

In [ ]:
torch.save(stuff, "/workdir/unsupervised_pretrain/jupyter/autoencoder-output.t")

# Load embeddings #

In [ ]:
device = torch.device("cpu")

In [ ]:
model = torch.load("/workdir/unsupervised_pretrain/model.pth", map_location=device).to(device)
model = model.eval()
autoencoder = torch.load("/workdir/unsupervised_pretrain/model-autoencoder.pth", map_location=device).to(device)
autoencoder = autoencoder.eval()

In [ ]:
_text_embeddings = torch.load("/workdir/unsupervised_pretrain/jupyter/text-embeddings.t")
text_embeddings = _text_embeddings.detach().cpu().numpy()

In [ ]:
_visual_embeddings = torch.load("/workdir/unsupervised_pretrain/jupyter/visual-embeddings.t")
visual_embeddings = _visual_embeddings.detach().cpu().numpy()

In [ ]:
_stuff = torch.load("/workdir/unsupervised_pretrain/jupyter/autoencoder-output.t")
stuff = [thing.detach().cpu().numpy() for thing in _stuff]

## 2D ##

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

### Visual embeddings ###

Blue dots are (projections) of original embeddings, orange dots are reconstructed by/from the autoencoder.

In [ ]:
tsne = TSNE(n_components=2, random_state=0)

data0 = tsne.fit_transform(visual_embeddings)
data1 = tsne.fit_transform(stuff[0])

# plot the result
plt.figure(figsize=(6, 5))
plt.scatter(data0[:, 0], data0[:, 1])
plt.scatter(data1[:, 0], data1[:, 1])
# plt.scatter(data_2d[[333], 0], data_2d[[333], 1])  # Wood
# plt.scatter(data_2d[[82], 0], data_2d[[82], 1])  # City
# plt.scatter(data_2d[[440], 0], data_2d[[440], 1])  # Water
plt.xlabel("t-SNE feature 0")
plt.ylabel("t-SNE feature 1")
plt.show()

In [ ]:
np.max(np.abs(np.mean(visual_embeddings, axis=1))), np.max(np.abs(np.mean(stuff[0], axis=1)))

In [ ]:
np.max(np.abs(np.mean(visual_embeddings - stuff[0], axis=1))), np.mean(np.abs(np.mean(visual_embeddings - stuff[0], axis=1)))

### Text embeddings ###

In [ ]:
tsne = TSNE(n_components=2, random_state=0)

mask = ~np.isnan(text_embeddings[:, 0])
data0 = tsne.fit_transform(text_embeddings[mask])
data1 = tsne.fit_transform(stuff[1][mask])

# plot the result
plt.figure(figsize=(6, 5))
plt.scatter(data0[:, 0], data0[:, 1])
plt.scatter(data1[:, 0], data1[:, 1])
# plt.scatter(data_2d[[333], 0], data_2d[[333], 1])  # Wood
# plt.scatter(data_2d[[82], 0], data_2d[[82], 1])  # City
# plt.scatter(data_2d[[440], 0], data_2d[[440], 1])  # Water
plt.xlabel("t-SNE feature 0")
plt.ylabel("t-SNE feature 1")
plt.show()

In [ ]:
mask = ~np.isnan(text_embeddings[:, 0])
np.max(np.abs(np.mean(text_embeddings[mask], axis=1))), np.max(np.abs(np.mean(stuff[1][mask], axis=1)))

In [ ]:
np.max(np.abs(np.mean(text_embeddings[mask] - stuff[1][mask], axis=1))), np.mean(np.abs(np.mean(text_embeddings[mask] - stuff[1][mask], axis=1)))

### Shared latent space ###

In [ ]:
tsne = TSNE(n_components=2, random_state=0)

mask = ~np.isnan(text_embeddings[:, 0])
data0 = tsne.fit_transform(stuff[2][mask])
data1 = tsne.fit_transform(stuff[3][mask])

# plot the result
plt.figure(figsize=(6, 5))
plt.scatter(data0[:, 0], data0[:, 1])
plt.scatter(data1[:, 0], data1[:, 1])
# plt.scatter(data_2d[[333], 0], data_2d[[333], 1])  # Wood
# plt.scatter(data_2d[[82], 0], data_2d[[82], 1])  # City
# plt.scatter(data_2d[[440], 0], data_2d[[440], 1])  # Water
plt.xlabel("t-SNE feature 0")
plt.ylabel("t-SNE feature 1")
plt.show()

In [ ]:
mask = ~np.isnan(text_embeddings[:, 0])
np.max(np.abs(np.mean(stuff[2][mask], axis=1))), np.max(np.abs(np.mean(stuff[3][mask], axis=1)))

In [ ]:
np.max(np.abs(np.mean(stuff[2][mask] - stuff[3][mask], axis=1))), np.mean(np.abs(np.mean(stuff[2][mask] - stuff[3][mask], axis=1)))

# Look for similarity #

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
from scipy import spatial
import numpy as np
import matplotlib.pyplot as plt

## Utility functions ##

In [ ]:
def top_k_query_cosine(query_vector, data, k):
    # calculate cosine similarities
    cosine_similarities = cosine_similarity(data, query_vector.reshape(1, -1)).flatten()

    # get top-k indices
    top_k_indices = np.argpartition(-cosine_similarities, k)[:k]
    
    # return indices of the top-k closest vectors
    return top_k_indices

In [ ]:
def top_k_query_l1(query_vector, data, k):
    # calculate L1 distances
    l1_distances = cdist(data, query_vector.reshape(1, -1), 'cityblock').flatten()

    # get top-k indices
    top_k_indices = np.argpartition(l1_distances, k)[:k]
    
    # return indices of the top-k closest vectors
    return top_k_indices

In [ ]:
def top_k_query_l2(query_vector, data, k):
    # calculate L2 distances
    l1_distances = cdist(data, query_vector.reshape(1, -1), 'euclidean').flatten()

    # get top-k indices
    top_k_indices = np.argpartition(l1_distances, k)[:k]
    
    # return indices of the top-k closest vectors
    return top_k_indices

In [ ]:
def display_image(images, image_number):
    # Check that image_number is valid
    if image_number < 0 or image_number >= images.shape[0]:
        raise ValueError('image_number must be between 0 and the number of images')

    # Get the RGB bands (adjusting for 1-based indexing)
    r = images[image_number, 3, :, :] # Red band
    g = images[image_number, 2, :, :] # Green band
    b = images[image_number, 1, :, :] # Blue band

    # Stack them along the last dimension to create an RGB image
    rgb = np.stack([r, g, b], axis=-1)

    # Clamp and scale to [0, 255] range for display
    rgb = np.clip(rgb, 0, 2500)  # Ensure values are within 0-2500
    rgb = (rgb / 2500) * 255  # Scale values to 0-255

    # Convert to 8-bit unsigned integer type
    rgb = rgb.astype(np.uint8)

    # Show the image
    plt.figure(figsize=(6, 6))
    plt.imshow(rgb)
    plt.axis('off')  # Hide the axes
    plt.show()

In [ ]:
def display_all_images(images):
    # Determine the grid size to accommodate all images
    grid_size = int(np.ceil(np.sqrt(images.shape[0])))

    fig, ax = plt.subplots(grid_size, grid_size, figsize=(12, 12))

    for i in range(grid_size * grid_size):
        if i < images.shape[0]:
            # Get the RGB bands (adjusting for 1-based indexing)
            r = images[i, 3, :, :]  # Red band
            g = images[i, 2, :, :]  # Green band
            b = images[i, 1, :, :]  # Blue band

            # Stack them along the last dimension to create an RGB image
            rgb = np.stack([r, g, b], axis=-1)

            # Clamp and scale to [0, 255] range for display
            rgb = np.clip(rgb, 0, 2500)  # Ensure values are within 0-2500
            rgb = (rgb / 2500) * 255  # Scale values to 0-255

            # Convert to 8-bit unsigned integer type
            rgb = rgb.astype(np.uint8)

            # Display the image
            ax[i // grid_size, i % grid_size].imshow(rgb)
            ax[i // grid_size, i % grid_size].axis('off')  # Hide the axes
        else:
            # Hide empty subplots
            ax[i // grid_size, i % grid_size].axis('off')

    plt.show()

## Visual-visual queries ##

### Water ###

In [ ]:
query_vector = visual_embeddings[440]
print(top_k_query_cosine(query_vector, visual_embeddings, 5))
print(top_k_query_l1(query_vector, visual_embeddings, 5))
print(top_k_query_l2(query_vector, visual_embeddings, 5))

In [ ]:
images_this = ds[440*2][0]
images_this = images_this.detach().cpu().numpy()
display_all_images(images_this)

In [ ]:
images_neighbor = ds[394*2][0]
images_neighbor = images_neighbor.detach().cpu().numpy()
display_all_images(images_neighbor)

In [ ]:
images_neighbor = ds[393*2][0]
images_neighbor = images_neighbor.detach().cpu().numpy()
display_all_images(images_neighbor)

In [ ]:
images_neighbor = ds[437*2][0]
images_neighbor = images_neighbor.detach().cpu().numpy()
display_all_images(images_neighbor)

In [ ]:
images_neighbor = ds[414*2][0]
images_neighbor = images_neighbor.detach().cpu().numpy()
display_all_images(images_neighbor)

### Farmland(?) ###

In [ ]:
query_vector = visual_embeddings[333]
print(top_k_query_cosine(query_vector, visual_embeddings, 5))
print(top_k_query_l1(query_vector, visual_embeddings, 5))
print(top_k_query_l2(query_vector, visual_embeddings, 5))

In [ ]:
images_this = ds[333*2][0]
images_this = images_this.detach().cpu().numpy()
display_all_images(images_this)

In [ ]:
images_neighbor = ds[313*2][0]
images_neighbor = images_neighbor.detach().cpu().numpy()
display_all_images(images_neighbor)

### Buildings ###

In [ ]:
query_vector = visual_embeddings[19 + 3*21]
print(top_k_query_cosine(query_vector, visual_embeddings, 5))
print(top_k_query_l1(query_vector, visual_embeddings, 5))
print(top_k_query_l2(query_vector, visual_embeddings, 5))

In [ ]:
images_this = ds[(19 + 3*21)*2][0]
images_this = images_this.detach().cpu().numpy()
display_all_images(images_this)

In [ ]:
images_neighbor = ds[104*2][0]
images_neighbor = images_neighbor.detach().cpu().numpy()
display_all_images(images_neighbor)

## Text-text queries ##

Currently using `instructor-xl` for this.  (The quality of these embeddings can more-or-less be taken as read 😉.)

### Water ###

In [ ]:
query_vector = text_embeddings[318]
print(top_k_query_cosine(query_vector, text_embeddings, 5))
print(top_k_query_l1(query_vector, text_embeddings, 5))
print(top_k_query_l2(query_vector, text_embeddings, 5))

- Chip 318 is described in the dataset this way: "Land use land cover: water."
- Chip 415: ditto
- Chip 225: ditto
- Chip 142: ditto
- Chip 143: ditto
- Chip 164: ditto

In [ ]:
images_this = ds[318*2][0]
images_this = images_this.detach().cpu().numpy()
display_all_images(images_this)

In [ ]:
images_neighbor = ds[415*2][0]
images_neighbor = images_neighbor.detach().cpu().numpy()
display_all_images(images_neighbor)

### Buildings ###

In [ ]:
query_vector = text_embeddings[19 + 3*21]
print(top_k_query_cosine(query_vector, text_embeddings, 5))
print(top_k_query_l1(query_vector, text_embeddings, 5))
print(top_k_query_l2(query_vector, text_embeddings, 5))

- Chip 82 (= 19 +3*21) is described in the dataset this way: "Buildings: numerous. "
- Chip 83: "Buildings: many. "
- Chip 61: "Buildings: many. "
- Chip 104: "Buildings: many. "
- Chip 81: "Buildings: many. "

In [ ]:
images_neighbor = ds[61*2][0]
images_neighbor = images_neighbor.detach().cpu().numpy()
display_all_images(images_neighbor)

In [ ]:
images_neighbor = ds[104*2][0]
images_neighbor = images_neighbor.detach().cpu().numpy()
display_all_images(images_neighbor)

## Neighborhood similarity ##

In [ ]:
def neighborhood_similarity(embeddings1, embeddings2, fn, k):
    intersections = []
    for embedding1, embedding2 in zip(embeddings1, embeddings2):
        neighborhood1 = set(fn(embedding1, embeddings1, k))
        neighborhood2 = set(fn(embedding2, embeddings2, k))
        intersections.append(len(neighborhood1 & neighborhood2))
    return (np.mean(intersections) - 1) / (k - 1)

In [ ]:
neighborhood_similarity(visual2shared_embeddings, text2shared_embeddings, top_k_query_cosine, 4)

In [ ]:
cosine_overlap = []
l1_overlap = []
for i in tqdm(range(2, 32)):
    # Don't Hassle Me I'm Local
    cosine_overlap.append(neighborhood_similarity(visual_embeddings, v2t_embeddings, top_k_query_cosine, i))

In [ ]:
plt.figure(figsize=(10, 2))
plt.plot(cosine_overlap)
plt.plot(l1_overlap)
plt.yticks([i * .20 for i in range(0, 5)])
plt.show()

In [ ]:
cosine_overlap = []
l1_overlap = []
for i in tqdm(range(2, 32)):
    cosine_overlap.append(neighborhood_similarity(ds_embeddings, v2t_embeddings, top_k_query_cosine, i))

In [ ]:
plt.figure(figsize=(10, 2))
plt.plot(cosine_overlap)
plt.plot(l1_overlap)
plt.yticks([i * .20 for i in range(0, 5)])
plt.show()

## Text queries ##

In [ ]:
from InstructorEmbedding import INSTRUCTOR
embed_model = INSTRUCTOR("hkunlp/instructor-xl").to(device)
embed_model.max_seq_length = 4096

In [ ]:
def text_visual_query(query_text, instruction, embeddings, k: int = 5):
    query = embed_model.encode([[instruction, query_text]])
    query = torch.from_numpy(query).to(device)
    with torch.inference_mode():
        _, z = autoencoder.autoencoder_2(query)
        z = z / z.norm(dim=1, keepdim=True)
        query = autoencoder.autoencoder_1.decoder(z)
        query = query / query.norm(dim=1, keepdim=True)
    query.detach().cpu().numpy()
    return top_k_query_cosine(query, embeddings, k)

In [ ]:
instruction = "Represent the geospatial data for retrieval; Input: "

In [ ]:
text_visual_query("Buildings: a few. Land use land cover: park.", instruction, stuff[0], 5)

In [ ]:
images_neighbor = ds[0*2][0]
images_neighbor = images_neighbor.detach().cpu().numpy()
display_all_images(images_neighbor)

In [ ]:
display_image(images_neighbor, 2)

# Scratch #

## Query 1 ##

In [ ]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [ ]:
query_embedding = embed_model.encode([[instruction, query]])
# text_query_embedding /= np.linalg.norm(text_query_embedding, keepdims=True)

In [ ]:
classifications = []

for i in range(visual_embeddings.shape[0]):
    visual = torch.from_numpy(visual_embeddings[[i]]).to(device)
    text = torch.from_numpy(query_embedding).to(device)
    with torch.inference_mode():
        result = classifier(visual, text).detach().cpu().numpy()
    classifications.append(result)

classifications = np.concatenate(classifications, axis=0)

In [ ]:
classifications

In [ ]:
sigmoid(classifications)

In [ ]:
np.argsort(classifications, axis=0)

In [ ]:
images_this = ds[377][0]
images_this = images_this.detach().cpu().numpy()
display_all_images(images_this)

In [ ]:
query_vector = visual_embeddings[314]
print(top_k_query_cosine(query_vector, visual_embeddings, 5))

## Query 2 ##

In [ ]:
instruction = "Represent the geospatial data for retrieval; Input: "
query = "Land use land cover: wood."
query_embedding = embed_model.encode([[instruction, query]])

In [ ]:
classifications = []

for i in range(visual_embeddings.shape[0]):
    visual = torch.from_numpy(visual_embeddings[[i]]).to(device)
    text = torch.from_numpy(query_embedding).to(device)
    with torch.inference_mode():
        result = classifier(visual, text).detach().cpu().numpy()
    classifications.append(result)

classifications = np.concatenate(classifications, axis=0)

In [ ]:
np.argsort(classifications, axis=0)

In [ ]:
images_this = ds[183*2][0]
images_this = images_this.detach().cpu().numpy()
display_all_images(images_this)

In [ ]:
len(stuff)

In [ ]:
_text_embeddings.device